In [7]:
import pandas as pd
import requests
import re
import time

headers = {
    'Referer': 'https://www.rottentomatoes.com/m/notebook/reviews?type=user',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.108 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
}

s = requests.Session()
        
def get_reviews(url):
    r = requests.get(url)
    movie_id = re.findall(r'(?<=movieId":")(.*)(?=","type)',r.text)[0]

    api_url = f"https://www.rottentomatoes.com/napi/movie/{movie_id}/reviews/user" #use reviews/userfor user reviews /criticsReviews/all for critics
    
    payload = {
        'direction': 'next',
        'endCursor': '',
        'startCursor': '',
    }
    
    review_data = []
    
    while True:
        r = s.get(api_url, headers=headers, params=payload)
        data = r.json()

        if not data['pageInfo']['hasNextPage']:
            break

        payload['endCursor'] = data['pageInfo']['endCursor']
        payload['startCursor'] = data['pageInfo']['startCursor'] if data['pageInfo'].get('startCursor') else ''

        review_data.extend(data['reviews'])
        time.sleep(1)
    
    return review_data

data = get_reviews('https://www.rottentomatoes.com/m/interstellar_2014/reviews')
df = pd.json_normalize(data)


KeyboardInterrupt: 

In [5]:
df.to_csv('output/reviews.csv', index=False)

In [3]:
print(df)

     creationDate  isFresh  isRotten  isRtUrl  isTop  \
0    Aug 22, 2022     True     False    False  False   
1    Jun 30, 2022     True     False    False  False   
2    May 27, 2022     True     False    False  False   
3    Feb 11, 2022     True     False    False  False   
4     Oct 9, 2021     True     False    False  False   
..            ...      ...       ...      ...    ...   
355  Oct 27, 2014     True     False    False   True   
356  Oct 27, 2014     True     False    False  False   
357  Oct 27, 2014    False      True    False  False   
358  Oct 27, 2014     True     False    False   True   
359  Oct 27, 2014     True     False    False   True   

                                             reviewUrl  \
0    https://keithandthemovies.com/2014/11/22/revie...   
1    https://deepfocusreview.com/reviews/interstellar/   
2    https://www.nextbestpicture.com/interstellar.html   
3    https://www.williamsonhomepage.com/community/w...   
4    https://www.nerdophiles.com/2014